In [1]:
import numpy as np
import torch
import pyepo

Auto-Sklearn cannot be imported.


In [2]:
import cvxpy as cvx
cvx.installed_solvers()

(CVXPY) Nov 21 03:27:55 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')
(CVXPY) Nov 21 03:27:55 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


['CLARABEL', 'COPT', 'ECOS', 'ECOS_BB', 'GUROBI', 'MOSEK', 'SCIPY', 'SCS']

## Data Set and Optimization Solver

In [3]:
# generate data
num_node = 20 # node size
num_data = 1000 # number of training data
num_feat = 10 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.tsp.genData(num_data+1000, num_feat, num_node, deg, e, seed=42)

In [4]:
from model import tspDFJModel
# set solver
optmodel = tspDFJModel(num_node)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
tour = optmodel.getTour(sol)
print(" -> ".join(map(str, tour)))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 57.6036
0 -> 5 -> 8 -> 10 -> 4 -> 12 -> 1 -> 17 -> 3 -> 16 -> 9 -> 18 -> 7 -> 2 -> 11 -> 13 -> 19 -> 6 -> 14 -> 15 -> 0


In [5]:
# split data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)

In [6]:
from dataset import optDatasetConstrs
# get training and test data set
dataset_train = optDatasetConstrs(optmodel, x_train, costs=c_train) # with binding constr
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, costs=c_test) # without binding constr

Optimizing for optDataset...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:20<00:00, 48.83it/s]


Optimizing for optDataset...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 105.67it/s]


In [7]:
# get training and test data set without costs
dataset_train = optDatasetConstrs(optmodel, x_train, sols=dataset_train.sols) # with binding constr

Obtaining constraints for optDataset...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 89.66it/s]


In [8]:
# get data loader
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## Prediction Model

In [9]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, num_node*(num_node-1)//2)

    def forward(self, x):
        out = self.linear(x)
        return out

## Train

In [10]:
lr = 5e-3
num_epochs = 1

In [11]:
import time
from tqdm import tqdm

def train(reg, ca_cos, lr, num_epochs):
    # set optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # running time
    elapsed = 0
    for epoch in range(num_epochs):
        tick = time.time()
        tbar = tqdm(loader_train)
        for data in tbar:
            x, w, t_ctr = data
            # forward pass
            cp = reg(x)
            loss = ca_cos(cp, t_ctr)
            # set log
            tbar.set_description("Loss: {:8.4f}".format(loss.item()))
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # record time
        tock = time.time()
        elapsed += tock - tick
    # regret
    regret = pyepo.metric.regret(reg, optmodel, loader_test)
    print("Regret: {:7.4f}%".format(regret*100))
    # elapsed
    print("Elapsed Time: {:.2f} Sec".format(elapsed))

### Gurobi Method 

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [13]:
import gurobipy as gp
from gurobipy import GRB

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        # ceate a model
        m = gp.Model("projection")
        # turn off output
        m.Params.outputFlag = 0
        # focus on numeric problem
        m.Params.NumericFocus = 3
        # varibles
        λ = m.addMVar(len(ctr), name="λ")
        # objective function
        obj = (cp - λ @ ctr) @ (cp - λ @ ctr)
        m.setObjective(obj, GRB.MINIMIZE)
        # solve
        m.optimize()
        # get solutions
        proj = λ.X @ ctr
        # normalize
        proj = torch.FloatTensor(proj / np.linalg.norm(proj))
        return proj

In [14]:
# init model
reg = LinearRegression()

In [15]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [16]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:16<00:00,  1.98it/s]


Regret: 138.3727%
Elapsed Time: 16.15 Sec


### Gurobi with Warm-Starting 

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [18]:
import gurobipy as gp
from gurobipy import GRB

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        # ceate a model
        m = gp.Model("projection")
        # turn off output
        m.Params.outputFlag = 0
        # focus on numeric problem
        m.Params.NumericFocus = 3
        # varibles
        λ = m.addMVar(len(ctr), name="λ")
        # warm-start
        init_λ = np.zeros(len(ctr))
        sign = ctr[-len(cp):].sum(axis=0)
        init_λ[-len(cp):] = np.abs(cp) * (np.sign(cp) == np.sign(sign)) # hyperqudrants
        λ.Start = init_λ
        # objective function
        obj = (cp - λ @ ctr) @ (cp - λ @ ctr)
        m.setObjective(obj, GRB.MINIMIZE)
        # solve
        m.optimize()
        # get solutions
        proj = λ.X @ ctr
        # normalize
        proj = torch.FloatTensor(proj / np.linalg.norm(proj))
        return proj

In [19]:
# init model
reg = LinearRegression()

In [20]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [21]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:17<00:00,  1.82it/s]


Regret: 138.3727%
Elapsed Time: 17.59 Sec


## Gurobi with CVXPY

In [22]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [23]:
import cvxpy as cvx

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        # varibles
        λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
        # onjective function
        objective = cvx.Minimize(cvx.sum_squares(cp - λ @ ctr))
        # ceate a model
        problem = cvx.Problem(objective)
        # solve and focus on numeric problem
        problem.solve(solver=cvx.GUROBI, solver_opts={"NumericFocus": 3})
        # get solutions
        proj = λ.value @ ctr
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [24]:
# init model
reg = LinearRegression()

In [25]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [26]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:27<00:00,  1.18it/s]


Regret: 138.3727%
Elapsed Time: 27.14 Sec


## COPT 

In [27]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [28]:
import os
import sys
from contextlib import contextmanager

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [29]:
from coptpy import Envr, EnvrConfig
from coptpy import COPT

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        with suppress_stdout():
            # no banner
            envconfig = EnvrConfig()
            envconfig.set("nobanner", "1")
            # ceate a model
            m = Envr(envconfig).createModel("projection")
            # turn off output
            m.setParam("Logging", 0)
            # focus on numeric problem
            m.setParam("NumericFocus", 3)
            # varibles
            λ = m.addMVar(len(ctr), nameprefix="λ")
            # onjective function
            obj = (cp - λ @ ctr) @ (cp - λ @ ctr)
            m.setObjective(obj, COPT.MINIMIZE)
            # solve
            m.solve()
        # get solutions
        proj = λ.X @ ctr
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [30]:
# init model
reg = LinearRegression()

In [31]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [32]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [02:37<00:00,  4.91s/it]


Regret: 138.3727%
Elapsed Time: 157.05 Sec


##  COPT with CVXPY

In [33]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [34]:
import cvxpy as cvx

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        with suppress_stdout():
            # varibles
            λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
            # onjective function
            objective = cvx.Minimize(cvx.sum_squares(cp - λ @ ctr))
            # ceate a model
            problem = cvx.Problem(objective)
            # solve and focus on numeric problem
            problem.solve(solver=cvx.COPT, NumericFocus=3)
        # get solutions
        proj = λ.value @ ctr
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [35]:
# init model
reg = LinearRegression()

In [36]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [37]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.53it/s]


Regret: 138.3727%
Elapsed Time: 20.99 Sec


## Mosek with CVXPY 

In [38]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [39]:
import cvxpy as cvx

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        # varibles
        λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
        # onjective function
        objective = cvx.Minimize(cvx.sum_squares(cp - λ @ ctr))
        # ceate a model
        problem = cvx.Problem(objective)
        # solve and focus on numeric problem
        problem.solve(solver=cvx.MOSEK)
        # get solutions
        proj = λ.value @ ctr
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [40]:
# init model
reg = LinearRegression()

In [41]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [42]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:18<00:00,  1.75it/s]


Regret: 138.3727%
Elapsed Time: 18.24 Sec


##  Clarabel

In [43]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [44]:
import clarabel as cl

from func import exactConeAlignedCosine

class ConeAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        # number of variable
        num_λ = ctr.shape[1]
        # quadratic term
        Q = ctr @ ctr.T
        # linear term
        p = - 2 * cp.T @ ctr
        # constraints λ >= 0
        A = - np.eye(num_λ)
        b = np.zeros(num_λ)
        # cone
        cones = [cl.NonnegativeConeT(num_λ)]
        # settings
        settings = cl.DefaultSettings()
        settings.verbose = False
        # QP model
        model = cl.DefaultSolver(Q, p, A, b, cones, settings)
        # solve
        result = model.solve()
        # get the solution
        proj = result.x[:num_p]
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [45]:
# init model
reg = LinearRegression()

In [46]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [47]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:16<00:00,  1.96it/s]


Regret: 138.3727%
Elapsed Time: 16.33 Sec


## Clarabel with CVXPY

In [48]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [49]:
import cvxpy as cvx

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        # varibles
        λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
        # onjective function
        objective = cvx.Minimize(cvx.sum_squares(cp - λ @ ctr))
        # ceate a model
        problem = cvx.Problem(objective)
        # solve and focus on numeric problem
        problem.solve(solver=cvx.CLARABEL)
        # get solutions
        proj = λ.value @ ctr
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [50]:
# init model
reg = LinearRegression()

In [51]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [52]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:10<00:00,  3.07it/s]


Regret: 138.3727%
Elapsed Time: 10.42 Sec


### Non-Negative Least Squares

In [53]:
# random seed
np.random.seed(42)
torch.manual_seed(42)

In [54]:
from scipy.optimize import nnls

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _solveQP(self, cp, ctr, warmstart, conecheck):
        # solve the linear equations
        λ, _ = nnls(ctr.T, cp)
        # get projection
        proj = λ @ ctr
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [55]:
# init model
reg = LinearRegression()

In [56]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

Num of cores: 1


In [57]:
train(reg, ca_cos, lr, num_epochs)

Loss:  -0.9531: 100%|██████████████████████████████████████████████████████████████████| 32/32 [00:11<00:00,  2.89it/s]


Regret: 138.3727%
Elapsed Time: 11.06 Sec
